In [1]:
import json
import random
from gensim import models
from random import shuffle
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from doc2vec.doc2vec import doc2vec
from node2vec.node2vec import node2vec

random.seed(42)
np.random.seed(42)

C:\Users\nebboy4\Anaconda3\envs\github_env\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Load Doc2Vec Model & Get Vectors

In [2]:
d2v_model = doc2vec(model=models.Doc2Vec.load("../data/scotus/scotus_model.doc2vec"),label_docs=False)
n2v_model = node2vec(model=models.Word2Vec.load("../data/scotus_n2v_1.0_1.0_mini.node2vec"))

In [3]:
d2v_embeddings = d2v_model.model.docvecs
n2v_embeddings = n2v_model.model.wv

In [4]:
data_d2v = json.load(open('../data/ia_doc2vec_clustering.json'))
data_n2v = json.load(open('../data/ia_node2vec_clustering_1.0_1.0.json'))

In [5]:
node_order_d2v = {}
node_ids_d2v = []
node_order_n2v = {}
node_ids_n2v = []

for i in range(len(data_d2v['node_ids'])):
    node_order_d2v[i] = data_d2v['node_ids'][i]
    node_ids_d2v.append(data_d2v['node_ids'][i])

for i in range(len(data_n2v['node_ids'])):
    node_order_n2v[i] = data_n2v['node_ids'][i]
    node_ids_n2v.append(data_n2v['node_ids'][i])
    
X_d2v = [d2v_embeddings[tag].tolist() for tag in node_ids_d2v]
X_n2v = [n2v_embeddings[tag].tolist() for tag in node_ids_n2v]

In [6]:
names_d2v = []
vectors_d2v = []
names_n2v = []
vectors_n2v = []

for tag,_ in d2v_embeddings.doctags.items():
    names_d2v.append(tag)
    vectors_d2v.append(d2v_embeddings[tag].tolist())

for tag,_ in n2v_embeddings.vocab.items():
    names_n2v.append(tag)
    vectors_n2v.append(n2v_embeddings[tag].tolist())
    
print('complete')

complete


In [7]:
name_to_ia = json.load(open('../data/name_to_ia_mini.json'))

In [34]:
mask_d2v = np.random.rand(len(names_d2v)) < 0.85
name_train_d2v = np.array(names_d2v)[mask_d2v]
name_test_d2v = np.array(names_d2v)[~mask_d2v]
mask_n2v = np.random.rand(len(names_n2v)) < 0.85
name_train_n2v = np.array(names_n2v)[mask_n2v]
name_test_n2v = np.array(names_n2v)[~mask_n2v]

X_train_d2v = []
Y_train_d2v = []
X_test_d2v = []
Y_test_d2v = []
X_train_n2v = []
Y_train_n2v = []
X_test_n2v = []
Y_test_n2v = []
X_train_both = {}
Y_train_both = {}
X_test_both = {}
Y_test_both = {}

for name,vec in zip(names_d2v,vectors_d2v):
    if name in name_train_d2v:
        X_train_d2v.append(vec)
        Y_train_d2v.append(int(name_to_ia[name]))
        X_train_both[name] = vec
        Y_train_both[name] = int(name_to_ia[name])
    elif name in name_test_d2v:
        X_test_d2v.append(vec)
        Y_test_d2v.append(int(name_to_ia[name]))
        X_test_both[name] = vec
        Y_test_both[name] = int(name_to_ia[name])
    else:
        print('D2V PROBLEM WITH:',name)

for name,vec in zip(names_n2v,vectors_n2v):
    if name in name_train_n2v:
        X_train_n2v.append(vec)
        Y_train_n2v.append(int(name_to_ia[name]))
    elif name in name_test_n2v:
        X_test_n2v.append(vec)
        Y_test_n2v.append(int(name_to_ia[name]))
    else:
        print('N2V PROBLEM WITH:',name)
    if name in name_train_d2v:
        X_train_both[name] = X_train_both[name] + vec
    elif name in name_test_d2v:
        X_test_both[name] = X_test_both[name] + vec

print(len(X_train_d2v),len(Y_train_d2v))
print(len(X_test_d2v),len(Y_test_d2v))
print(len(X_train_n2v),len(Y_train_n2v))
print(len(X_test_n2v),len(Y_test_n2v))
print(len(X_train_both),len(Y_train_both))
print(len(X_test_both),len(Y_test_both))

23762 23762
4123 4123
23774 23774
4111 4111
23762 23762
4123 4123


In [35]:
train_names = list(X_train_both.keys())
X_train_both = [X_train_both[n] for n in train_names]
Y_train_both = [Y_train_both[n] for n in train_names]

test_names = list(X_test_both.keys())
X_test_both = [X_test_both[n] for n in test_names]
Y_test_both = [Y_test_both[n] for n in test_names]

In [20]:
d2v_log_reg = LogisticRegressionCV([.01,.1,1,10,100],
                                   penalty='l2',
                                   scoring='accuracy',
                                   solver='sag',
                                   n_jobs=-1,
                                   verbose=1,
                                   refit=True,
                                   multi_class='multinomial'
                                   )

n2v_log_reg = LogisticRegressionCV([.01,.1,1,10,100],
                                   penalty='l2',
                                   scoring='accuracy',
                                   solver='sag',
                                   n_jobs=-1,
                                   verbose=1,
                                   refit=True,
                                   multi_class='multinomial'
                                   )

In [36]:
both_log_reg = LogisticRegressionCV([.01,.1,1,10,100],
                                   penalty='l2',
                                   scoring='accuracy',
                                   solver='sag',
                                   n_jobs=-1,
                                   verbose=1,
                                   refit=True,
                                   multi_class='multinomial'
                                   )

In [25]:
d2v_log_reg.fit(X_train_d2v, Y_train_d2v)

convergence after 63 epochs took 25 seconds
convergence after 64 epochs took 27 seconds
convergence after 65 epochs took 28 seconds
max_iter reached after 49 seconds
max_iter reached after 51 seconds
max_iter reached after 51 seconds
max_iter reached after 50 seconds
max_iter reached after 53 seconds
max_iter reached after 54 seconds
max_iter reached after 49 seconds
max_iter reached after 48 seconds
max_iter reached after 49 seconds
max_iter reached after 44 seconds
max_iter reached after 44 seconds
max_iter reached after 43 seconds


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.8min finished


LogisticRegressionCV(Cs=[0.01, 0.1, 1, 10, 100], class_weight=None, cv=None,
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='multinomial', n_jobs=-1,
           penalty='l2', random_state=None, refit=True, scoring='accuracy',
           solver='sag', tol=0.0001, verbose=1)

In [26]:
n2v_log_reg.fit(X_train_n2v, Y_train_n2v)

convergence after 21 epochs took 9 seconds
convergence after 20 epochs took 9 seconds
convergence after 21 epochs took 9 seconds
convergence after 39 epochs took 16 seconds
convergence after 39 epochs took 17 seconds
convergence after 40 epochs took 18 seconds
convergence after 77 epochs took 33 seconds
convergence after 77 epochs took 36 seconds
convergence after 76 epochs took 35 seconds
max_iter reached after 43 seconds
max_iter reached after 45 seconds
max_iter reached after 45 seconds
max_iter reached after 42 seconds
max_iter reached after 48 seconds
max_iter reached after 49 seconds


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.6min finished


LogisticRegressionCV(Cs=[0.01, 0.1, 1, 10, 100], class_weight=None, cv=None,
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='multinomial', n_jobs=-1,
           penalty='l2', random_state=None, refit=True, scoring='accuracy',
           solver='sag', tol=0.0001, verbose=1)

In [37]:
both_log_reg.fit(X_train_both, Y_train_both)

convergence after 81 epochs took 76 seconds
convergence after 83 epochs took 77 seconds
convergence after 84 epochs took 80 seconds
max_iter reached after 94 seconds
max_iter reached after 95 seconds
max_iter reached after 96 seconds
max_iter reached after 87 seconds
max_iter reached after 89 seconds
max_iter reached after 91 seconds
max_iter reached after 89 seconds
max_iter reached after 91 seconds
max_iter reached after 93 seconds
max_iter reached after 118 seconds
max_iter reached after 119 seconds
max_iter reached after 120 seconds


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  8.0min finished


LogisticRegressionCV(Cs=[0.01, 0.1, 1, 10, 100], class_weight=None, cv=None,
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='multinomial', n_jobs=-1,
           penalty='l2', random_state=None, refit=True, scoring='accuracy',
           solver='sag', tol=0.0001, verbose=1)

In [38]:
import pickle
pickle.dump(d2v_log_reg, open('../data/d2v_log_reg_first.pickle', 'wb'))
pickle.dump(n2v_log_reg, open('../data/n2v_log_reg_first.pickle', 'wb'))
pickle.dump(both_log_reg, open('../data/both_log_reg_first.pickle', 'wb'))

In [39]:
print(d2v_log_reg.score(X_test_d2v,Y_test_d2v))

0.695367450885


In [40]:
print(n2v_log_reg.score(X_test_n2v,Y_test_n2v))

0.58088056434


In [41]:
print(both_log_reg.score(X_test_both,Y_test_both))

0.686393402862


In [51]:
importance = sorted([_ for _ in range(len(both_log_reg.coef_[0]))], key = lambda x: both_log_reg.coef_[0][x])
print('Proportion of first 300 sorted coefficients that are doc2vec\'s:',sum([1 for i in importance[:300] if i<300]))
print('Proportion of first 300 sorted coefficients that are node2vec\'s:',sum([1 for i in importance[:300] if i>=300]))

Proportion of first 300 sorted coefficients that are doc2vec's: 154
Proportion of first 300 sorted coefficients that are node2vec's: 146


In [43]:
TESTING = False
if TESTING:
    x = [[random.uniform(0,100) for _ in range(300)] for _i_ in range(3000)]
    y = [int(random.uniform(0,1) > 0.5) for _ in range(3000)]
    a = LogisticRegressionCV([.01,.1,1,10,100],
                             penalty='l2',
                             scoring='accuracy',
                             solver='sag',
                             n_jobs=-1,
                             verbose=1,
                             refit=True,
                             multi_class='multinomial'
                            )
    a.fit(x,y)